In [2]:

DATA_PREPARATION_VOTE_METHOD = "sum_and_normalize" 

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import sys

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
    sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.utils.utils import get_models_save_path, get_submission_csv_path, running_in_kaggle, get_raw_data_dir
from src.utils.constants import Constants
from src.datasets.eeg_dataset import EEGDataset
from src.models.gru_convolution1d import GRUConvModel


In [3]:
DATA_PATH = get_raw_data_dir()
class InfCFG:
    model_name = 'GRUConv'
    hidden_units = 256
    num_layers = 3
    target_size = 6
    num_channels = 20
    downsample_factor = 2
    
    data_path = DATA_PATH
    n_splits = 5
    
    batch_size = 32
    num_workers = 0
    
    if running_in_kaggle():
        MODEL_DIR = f'/kaggle/input/gru-conv-sum-votes/'
    else:
        MODEL_DIR = get_models_save_path() / "GRUConvModel" / DATA_PREPARATION_VOTE_METHOD

InfCFG.model_paths = [os.path.join(InfCFG.MODEL_DIR, f'best_model_fold{i}.pth') for i in range(InfCFG.n_splits)]

TARGETS = Constants.TARGETS




In [4]:
DATA_PATH

PosixPath('/home/maiko/Documents/HSLU/AICOMP/HSLU.AICOMP.HMS/data')

In [5]:
def run_inference():
    """Executes the main inference loop."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    test_df = pd.read_csv(os.path.join(InfCFG.data_path, 'test.csv'))
    
    test_dataset = EEGDataset(df=test_df, data_path=InfCFG.data_path, mode='test', downsample_factor=InfCFG.downsample_factor)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=InfCFG.batch_size, 
        shuffle=False, 
        num_workers=InfCFG.num_workers
    )

    all_fold_predictions = []

    for i, path in enumerate(InfCFG.model_paths):
        print(f"\n========== Inferencing with Fold {i} Model ==========")
        if not os.path.exists(path):
            print(f"Model file not found: {path}. Skipping this fold.")
            continue
            
        model = GRUConvModel(
            input_size=InfCFG.num_channels,
            hidden_size=InfCFG.hidden_units,
            num_layers=InfCFG.num_layers,
            num_classes=InfCFG.target_size
        )
        model.load_state_dict(torch.load(path, map_location=device))
        model.to(device)
        model.eval()

        current_fold_preds = []
        with torch.no_grad():
            for signals in tqdm(test_loader, desc=f"Predicting Fold {i}"):
                outputs = model(signals.to(device))
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                current_fold_preds.append(probs)
        
        all_fold_predictions.append(np.concatenate(current_fold_preds))

    if not all_fold_predictions:
        print("No models were found for inference. Aborting.")
        return

    avg_predictions = np.mean(all_fold_predictions, axis=0)
    
    submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
    submission[TARGETS] = avg_predictions
    submission.to_csv(get_submission_csv_path(), index=False)
    

    print(submission.head())

if __name__ == '__main__':
    run_inference()

Using device: cuda

========== Inferencing with Fold 0 Model ==========


Predicting Fold 0:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 1 Model ==========


Predicting Fold 1:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 2 Model ==========


...


========== Inferencing with Fold 3 Model ==========


Predicting Fold 3:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 4 Model ==========


Predicting Fold 4:   0%|          | 0/1 [00:00<?, ?it/s]

       eeg_id  seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  \
0  3911565283      0.068829  0.031743  0.002197   0.103908   0.039986   

   other_vote  
0    0.753337  
